In [4]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [3]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikiforova\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [6]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [7]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [8]:
data = pd.concat([pd.read_json(file, lines=True,  encoding='utf-8') for file in files][:1], axis=0, ignore_index=True)
# Здесь было необходимо обозначить кодировку UTF-8, иначе по дефолту считывал в ANSI и вместо кириллицы отображал каракули

In [9]:
data.shape

(999, 5)

In [10]:
data.head(3)

,content,keywords,summary,title,url
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html
1,"Вчера «Газпром» снизил верхнюю планку прогноза собственной добычи газа в 2020 году. Через 12 лет концерн собирается добывать около 620–640 млрд. куб. м в год. При этом общее производство газа в стране, по расчетам холдинга, должно достичь 940 млрд. куб. м. Иными словами, треть добываемого объема, по мнению холдинга, должны будут обеспечить независимые производители. Эксперты не верят, что независимые компании смогут выйти на такие объемы добычи. Если расчеты «Газпрома» не оправдаются, то под ударом окажутся отечественные предприятия и население, которым придется сокращать потребление и смириться с новым витком цен. Иных путей покрытия возможного дефицита газа нет, так как вряд ли холдинг разорвет уже заключенные контракты на экспорт газа в другие страны. \n«Газпром» к 2020 году планирует добывать 620–640 млрд. куб. м газа, сообщил вчера на форуме «ТЭК России в ХХI веке» глава управления по добыче газа, газового конденсата и нефти холдинга Валерий Минликаев. Тем самым он уточнил пре...","[газпром, газ]",,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazprom.html
2,"Долголетний труд Евгения Витковского на ниве перевода, а также в качестве редактора и антологиста известен многим. Но не все знают его как поэта и прозаика. В этом году уже вышла составленная им и Еленой Кистеровой антология «Раздол туманов: Страницы шотландской гэльской поэзии XVII–XX вв.», а в апреле запланирован выход его романа «Протей, или Византийский кризис» (отрывок из романа читайте на с. 12). С \n побеседовал \n– Одна из таких книг только что вышла – «Раздол туманов. Страницы шотландской гэльской поэзии XVII–XX веков». Это стихи 29 поэтов, все в переводе с оригинала – моем и Елены Кистеровой. Работа заняла 10 лет, включая изучение языка. Она была упоительно интересной: до нас переводов из этой поэзии на русский не было вовсе. Сейчас должен выйти том стихотворений канадского классика Роберта Уильяма Сервиса, «канадского Киплинга», около 300 стихотворений. Кроме того, в Петербурге в производстве наш огромный трехтомный плод совместной работы – антология «Франция в сердце»....","[франсуа рабле, сервантес, шекспир, конан дойл, михаил булгаков, александр грин, борхес, босх, маркес, герман гессе, голландская живопись, гаргантюа и пантагрюэль, дон кихот, мастер и маргарита, москва, россия, история, поэзия, шотландия, баллада, пере]","Евгений Витковский о том, как Босх протягивает руку Шекспиру, \r\nи оба танцуют в пламени пожара в охваченном чумой средневековом городе",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitkovsky.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [13]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [14]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [22]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5])) 

Precision -  0.06
Recall -  0.06
F1 -  0.06
Jaccard -  0.03


Или 10.

In [23]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.07
F1 -  0.06
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [24]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [25]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [26]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [27]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                      [в, и, на, не, что, –, его, «молодежное, с, это]
1                                                            [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                                 [в, –, и, не, я, но, что, это, на, с]
3                                                       [в, на, и, ким, по, –, что, видео, он, зинаиды]
4                                              [в, и, на, новой, площадью, москвы, –, развития, с, для]
5                                                             [в, на, и, не, с, но, уже, что, у, гонки]
6                                                  [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                                        [в, и, –, по, с, культуре, не, из, будет, как]
8                                                              [в, и, на, с, что, для, по, –, не, газа]
9    [в, рф, террористической, организации, задержаны, –, четвер

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [46]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [47]:
stop_names_female = open('stop_names_female.txt', encoding = 'utf-8').readlines()
stop_names_male = open('stop_names_male.txt', encoding = 'utf-8').readlines()

stop_names = stop_names_male + stop_names_female
stop_names = [line.rstrip('\n').lower() for line in stop_names]

In [48]:
def normalize_1(text):
    # + remove names and restrics to 3+ letters in a word
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
    words = [word for word in words if word not in stop_names] # избавимся от дарьи, вечно она лезет куда не надо
    words = [word for word in words if len(word) > 2] # в данных попадаются га (гектары), км (километры) и т.п.

    return words

In [49]:
data['content_norm'] = data['content'].apply(normalize) # на этих данных был достигнут бейзлайн

In [50]:
data['content_nonames'] = data['content'].apply(normalize_1) # то же, что и 'content_norm', но без имен и без слов короче 3

In [51]:
data['title_norm'] = data['title'].apply(normalize)

In [52]:
data['title_nonames'] = data['title'].apply(normalize_1)

In [56]:
data['title_norm'][252:262]

252                             [юлия, тимошенко, замена, выбор]
253                                                     [солнце]
254    [встреча, соперник, выбор, путин, состав, новое, кабмина]
255                                       [порошенко, электорат]
256                                         [избиение, младенец]
257                                               [нефть, рубль]
258                                           [игра, джентльмен]
259                 [фестиваль, театр, ребёнок, карабас, москва]
260                                                     [байкал]
261                                                 [риск, дело]
Name: title_norm, dtype: object

In [57]:
data['title_nonames'][252:262]

252                                   [тимошенко, замена, выбор]
253                                                     [солнце]
254    [встреча, соперник, выбор, путин, состав, новое, кабмина]
255                                       [порошенко, электорат]
256                                         [избиение, младенец]
257                                               [нефть, рубль]
258                                           [игра, джентльмен]
259                 [фестиваль, театр, ребёнок, карабас, москва]
260                                                     [байкал]
261                                                 [риск, дело]
Name: title_nonames, dtype: object

_____________
**NB!** Причем лемматизирует очень качественно (за исключением многозначных слов). 

Например, видела тут пример, где слово "парка" вместо того, чтобы быть лемматизированным до "парк", осталось таким же. Потому что парка (а-ля куртка) тоже существует.
Возможно, по этой причине есть смысл попробовать стемминг...
_____________

Попробуем те же самые методы.

In [58]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


In [59]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.2
Recall -  0.13
F1 -  0.14
Jaccard -  0.09


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [60]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1                       [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                        [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                             [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                           [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                            [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6                       [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7            [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                            [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                         [участник, рф, орган

In [42]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1               [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                     [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                   [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                    [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                    [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                 [участник, рф, организация, государство, область, центр, связь, фсб, март, год]
Name: content_norm, 

In [70]:
evaluate(data['keywords'], data['content_nonames'].apply(lambda x: [x[0] for x in Counter(x).most_common(9)]))

Precision -  0.14
Recall -  0.24
F1 -  0.17
Jaccard -  0.1


**Ура! +1 к F-score!**

Precision -  0.14
Recall -  0.24
F1 -  0.17
Jaccard -  0.1

In [78]:
evaluate(data['keywords'],data['title_nonames'].apply(lambda x: x[:10]))

Precision -  0.2
Recall -  0.13
F1 -  0.14
Jaccard -  0.09


Тут без продвижений :(

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [79]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [81]:
data['content_nonames_str'] = data['content_nonames'].apply(' '.join)

In [82]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [83]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [84]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [85]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [86]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [87]:
print(keywords[:3])

[['яблоко', 'активист', 'акция', 'дарья', 'деятельность', 'молодая человек', 'политика', 'виктор', 'выборы', 'силовик'], ['миллиард куб', 'куб метр', 'куб', 'газпром', 'газа', 'миллиард', 'добыча', 'метр', 'добыча газа', 'холдинг'], ['роман', 'книга', 'жанр', 'стихотворение', 'перевод', 'читатель', 'год', 'поэзия', 'произведение', 'том']]


In [88]:
evaluate(data['keywords'], keywords)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


___________
### Попытки побить бейзлайн

In [19]:
# stop_names[655:660] # допустим

['александра', 'александрина', 'алексина', 'алёна', 'алина']

In [107]:
# можно заодно сделать нграммы
tfidf_nonames = TfidfVectorizer(ngram_range=(1,3), min_df=5, max_df=800, max_features=5000)

In [108]:
tfidf_nonames.fit(data['content_nonames_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=800, max_features=5000, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [109]:
id2word = {i:word for i,word in enumerate(tfidf_nonames.get_feature_names())}

In [110]:
texts_vectors = tfidf_nonames.transform(data['content_nonames_str'])

In [111]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords_nonames = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

In [112]:
evaluate(data['keywords'], keywords_nonames)

Precision -  0.17
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


**Ура! Совсем немного, но прошли бейзлайн!**

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1

У нас же еще есть заголовки! А что будет, если их присоединить к тексту статей? Они ведь тоже несут важную информацию.

In [115]:
all_data = []
for i in data['content_norm_str'][:-1], ['title_norm']:
    # исключили один элемент в 'content_norm_str', он содержит ненужную нам строку
    all_data.extend(i)

In [117]:
tfidf = TfidfVectorizer(ngram_range=(1,3), min_df=5, max_df=800, max_features=5000)
tfidf.fit(all_data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=800, max_features=5000, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [118]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [119]:
texts_vectors = tfidf.transform(all_data)

In [120]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords_names = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

In [132]:
print(keywords_names[:3])

[['яблоко', 'активист', 'акция', 'дарья', 'деятельность'], ['миллиард куб', 'миллиард куб метр', 'куб метр', 'куб', 'газпром'], ['роман', 'книга', 'жанр', 'стихотворение', 'читатель']]


In [126]:
evaluate(data['keywords'], keywords_names) # Precision +0.1!

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


О, +1 к  Precision! На первоначально нормализованных данных.

Давайте посмотрим, что с данными без имен.

In [127]:
all_data_nonames = []
for i in data['content_nonames_str'][:-1], ['title_nonames']:
    # исключили один элемент в 'content_norm_str', он содержит ненужную нам строку
    all_data_nonames.extend(i)

In [128]:
tfidf_nonames = TfidfVectorizer(ngram_range=(1,3), min_df=5, max_df=800, max_features=5000)
tfidf_nonames.fit(all_data_nonames)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=800, max_features=5000, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [129]:
id2word = {i:word for i,word in enumerate(tfidf_nonames.get_feature_names())}

In [130]:
texts_vectors = tfidf_nonames.transform(all_data_nonames)

In [131]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords_nonames = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

In [133]:
print(keywords_nonames[:3])

[['яблоко', 'активист', 'акция', 'деятельность', 'молодая человек'], ['миллиард куб', 'миллиард куб метр', 'куб метр', 'куб', 'газпром'], ['книга', 'жанр', 'стихотворение', 'читатель', 'перевод']]


Благополучно выкинули Дарью

In [134]:
evaluate(data['keywords'], keywords_nonames)

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


Этот вариант тоже остается в +0.1 к F-score бейзлайна, но, к сожалению, он не лучше корпуса без имен.

Что если увеличить количество ключевых слов?

In [166]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords_nonames = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 

In [167]:
print(keywords_nonames[:3])

[['яблоко', 'активист', 'акция', 'деятельность', 'молодая человек', 'политика'], ['миллиард куб', 'миллиард куб метр', 'куб метр', 'куб', 'газпром', 'газа'], ['книга', 'жанр', 'стихотворение', 'читатель', 'перевод', 'поэзия']]


In [168]:
evaluate(data['keywords'], keywords_nonames)

Precision -  0.16
Recall -  0.19
F1 -  0.17
Jaccard -  0.1


Получаем + к recall, но - precision, f-score без изменений.

_____
#### Итак, что было сделано:

* **most_common(9)** попробовали выделять не 10, а 9 ключевых слов. 

Precision -  0.14
Recall -  0.24
F1 -  0.17
Jaccard -  0.1

* **Избавились от имен в данных.** Причем, Собянин, Путин и тимошенко не исчезнли. Только имена, которые ни о чем не говорят
* **ngram_range=(1,3)**, т.к. в ключах может быть что-нибудь типа "мастера и маргариты"
* **max_df=800, max_features=5000** - таким образом мы отсеяли "кастомные" стоп-слова, указав интервал частоты встречаемости слов в документах и частоту употребления во всем корпусе. Я пробовала много разных вариаций этих параметров, несколько раз пришлось ребутить кернел :( Эти параметры работают в принципе хорошо. Возможные интервалы, которые не влияют на общий резцльтат: max_df=[650:850], max_features=[3500:6500]
* **ввели условие len(word) > 2** - до этого в ключевые слова попадали такие слова как га (гектар), км и проч.
* **texts_vectors_all.toarray().argsort()[:,:-6:-1]** Вместо 10 ищем 5 ключевых слов, поскольку в размеченном руками корпусе зачастую не очень много ключевых слов. Таким образом, мы будем меньше попадать мимо. Но это скажется на полноте (recall), что логично.

В итоге выросли все метрики (чуть-чуть), кроме recall. 

Precision -  0.17
Recall -  0.18
F1 -  0.17
Jaccard -  0.1

* **дополнительная нормализация заголовков** и попытка извлечения ключевых слов из заголовков

Тут получилось немного повысить Precision, и в принципе результат лучше, чем до дополнительной нормализации. Но до бейзлайна не дотягивает

Precision -  0.2
Recall -  0.13
F1 -  0.14
Jaccard -  0.09

* **конкатенация текстов с их заголовками** - а почему нет? заголовки все-таки несут важную информацию, их тоже стоит учитывать

Precision выросла на 0.1 по сравнению с первым результатом.

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1

______________

***...Продолжение заметок автора...***

Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [169]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [211]:
def get_kws(text, top=6, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [212]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 8, 8))

Wall time: 36.9 s


In [213]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.12
Recall -  0.18
F1 -  0.14
Jaccard -  0.08


In [214]:
keywords_rw.head(10)

0                     [яблоко, год, деятельность, активист, жизнь, клуб, тема, акция]
1                           [газа, куб, год, миллиард, газпром, страна, метр, добыча]
2                 [книга, год, произведение, стихотворение, том, москва, мир, россия]
3           [ким, размер, зинаида, журналист, видео, качество, свидетель, экспертиза]
4                      [москва, территория, га, площадь, развитие, тинао, год, парка]
5                  [гонка, место, позиция, команда, пилот, чемпионат, строчка, вернь]
6                  [место, рф, глава, позиция, россия, исследование, сергей, влияние]
7    [культура, закон, услуга, изменение, сообщество, учреждение, библиотекарь, дело]
8                      [газопровод, сирия, россия, турция, европа, иран, катар, газа]
9    [деятельность, рф, организация, прошедшее, сирия, подготовка, данные, сообщение]
Name: content_norm, dtype: object

In [215]:
%%time
keywords_rw = data['content_nonames'].apply(lambda x: get_kws(x, 7, 7))

Wall time: 37.4 s


In [216]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.12
Recall -  0.17
F1 -  0.14
Jaccard -  0.08


In [217]:
keywords_rw.head(10)

0        [деятельность, активист, россия, яблоко, год, обязательство, акция]
1                   [метр, куб, миллиард, газа, газпром, год, производитель]
2                       [книга, год, мир, вид, произведение, рука, читатель]
3           [видео, участок, суд, приморье, вознаграждение, рубль, 25-летие]
4                  [территория, парк, москва, площадь, год, регион, столица]
5                 [гонка, место, позиция, команда, конструктор, кубок, бокс]
6                [место, позиция, влияние, глава, эксперт, опрос, сообщение]
7                  [культура, проект, закон, сфера, кодекс, первое, система]
8                  [год, газопровод, турция, россия, сирия, европа, военный]
9    [территория, подготовка, данные, сирия, сообщение, участник, прошедшее]
Name: content_nonames, dtype: object

На глаз видно, что результат без имен гораздо лучше. 
Но, так как он все же отличается от человеческой разметки, он не может превзойти безлайн

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [218]:
import networkx as nx

In [220]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=4, topn=6):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_matrix(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

**NB!** in networkx 2.2 **```from_numpy_array```** was renamed to **```from_numpy matrix```**

In [230]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 9, 9))

Wall time: 1min 22s


In [231]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.23
F1 -  0.16
Jaccard -  0.09


In [232]:
keyword_nx[:2]

0    [яблоко, акция, год, активист, политика, деятельность, власть, дарья, жизнь]
1       [газа, миллиард, год, газпром, куб, метр, добыча, производитель, холдинг]
Name: content_norm, dtype: object

In [233]:
%%time
keyword_nx = data["content_nonames"].apply(lambda x: some_centrality_measure(x, 9, 9))

Wall time: 1min 19s


In [234]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.23
F1 -  0.16
Jaccard -  0.09


In [235]:
keyword_nx[:2]

0    [яблоко, акция, год, активист, политика, деятельность, власть, жизнь, россия]
1         [миллиард, газа, год, куб, газпром, метр, добыча, производитель, страна]
Name: content_nonames, dtype: object

Получилось достигнуть F1 -  0.16, дальше почему-то не идет :(
__________

Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [238]:
from gensim.summarization import keywords

C:\Users\Nikiforova\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [239]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:8])

In [240]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.1
F1 -  0.08
Jaccard -  0.05


Попробуем на корпусе без имен.

In [246]:
gensim_nonames = data['content_nonames'].apply(lambda x: keywords(' '.join(x)).split('\n')[:7])

In [247]:
evaluate(data['keywords'], gensim_nonames)

Precision -  0.08
Recall -  0.1
F1 -  0.08
Jaccard -  0.05


In [248]:
gensim_nonames[:3]

0                [яблоко, жизнь год, политика, акция уровень борьба, украина активист, власть, россия]
1    [газпром, миллиард, прогноз добыча газа год, страна, метр, производитель эксперт, расчет холдинг]
2                         [книга, поэт прозаик год, читатель, мир, герои, произведение, жанр мениппея]
Name: content_nonames, dtype: object

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).